# Validação Cruzada com KNN no Conjunto de Dados de Vinhos
Este notebook utiliza o método **k-fold cross-validation** com o classificador **K-Nearest Neighbors (KNN)** para avaliar o desempenho do modelo com diferentes valores de `k` e `random_state`.

### Objetivos:
1. Avaliar o impacto de diferentes valores de `k` (3 e 5) na acurácia.
2. Explorar como diferentes valores de `random_state` influenciam os resultados.
3. Aplicar validação cruzada para obter métricas robustas de avaliação.


In [2]:
#Importação das bibliotecas

import pandas as pd
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np


In [3]:
#Carregamento e pré-processamento de dados
# URL do dataset Wine
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

# Nome das colunas em português BR
column_names_pt_br = [
    'classe',
    'alcool',
    'acido_malico',
    'cinzas',
    'alcalinidade_de_cinzas',
    'magnesio',
    'fenois_totais',
    'flavanoides',
    'fenois_nao_flavanoides',
    'proantocianinas',
    'intensidade_de_cor',
    'matiz',
    'od280_od315_de_vinhos_diluidos',
    'prolina'
]

# Carregar os dados
vinhos = pd.read_csv(url, names=column_names_pt_br, dtype={'classe': object})

# Separação das variáveis
X = vinhos.drop('classe', axis=1)  # Variáveis preditoras
y = vinhos['classe']              # Variável alvo


In [4]:
#Função de K-Fold Cross-Validation
def kfold_validation(X, y, k_values, random_states, n_splits=10):
    resultados = []

    for random_state in random_states:
        print(f"\n== Random State: {random_state} ==")
        # Configuração do KFold
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

        for k in k_values:
            acuracias = []  # Lista para armazenar as acurácias de cada fold

            for train_index, test_index in kf.split(X):
                # Separação dos dados em treino e teste para cada fold
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                # Treinamento do modelo KNN
                knn = KNeighborsClassifier(n_neighbors=k)
                knn.fit(X_train, y_train)

                # Previsão e avaliação
                y_pred = knn.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                acuracias.append(accuracy)

            # Calcula média e desvio padrão das acurácias
            media = np.mean(acuracias)
            desvio = np.std(acuracias)
            resultados.append({'random_state': random_state, 'k': k, 'media': media, 'desvio': desvio})

            print(f"k={k}, Média da Acurácia: {media:.4f}, Desvio Padrão: {desvio:.4f}")

    return pd.DataFrame(resultados)


In [5]:
#Execução do experimento

# Configuração dos valores de k e random_state
k_values = [3, 5]
random_states = [42, 17, 24]

# Executa a validação cruzada
resultados = kfold_validation(X, y, k_values, random_states)



== Random State: 42 ==
k=3, Média da Acurácia: 0.7036, Desvio Padrão: 0.1392
k=5, Média da Acurácia: 0.6641, Desvio Padrão: 0.1001

== Random State: 17 ==
k=3, Média da Acurácia: 0.6987, Desvio Padrão: 0.1040
k=5, Média da Acurácia: 0.6876, Desvio Padrão: 0.1121

== Random State: 24 ==
k=3, Média da Acurácia: 0.7127, Desvio Padrão: 0.0968
k=5, Média da Acurácia: 0.7127, Desvio Padrão: 0.0940


In [6]:
#Análise Final

# Melhor configuração por random_state
print("\n== Análise Final ==")
for random_state in random_states:
    subset = resultados[resultados['random_state'] == random_state]
    melhor_k = subset.loc[subset['media'].idxmax()]
    print(f"Random State: {random_state}, Melhor k: {melhor_k['k']}, Média: {melhor_k['media']:.4f}, Desvio: {melhor_k['desvio']:.4f}")



== Análise Final ==
Random State: 42, Melhor k: 3.0, Média: 0.7036, Desvio: 0.1392
Random State: 17, Melhor k: 3.0, Média: 0.6987, Desvio: 0.1040
Random State: 24, Melhor k: 3.0, Média: 0.7127, Desvio: 0.0968
